In [ ]:
import os
import time
import json
import math
import logging
import argparse
from dataclasses import dataclass, asdict
from typing import List, Dict, Tuple, Optional

import numpy as np
import networkx as nx
import matplotlib.pyplot as plt
import matplotlib.cm as cm

from qiskit import QuantumCircuit
from qiskit.quantum_info import Statevector
from qiskit_aer import AerSimulator
from scipy.optimize import minimize
from scipy.ndimage import uniform_filter1d

# ------------------------- Logging -------------------------

def setup_logger(name: str = "qaoa", level: int = logging.INFO) -> logging.Logger:
    logger = logging.getLogger(name)
    if not logger.handlers:
        logger.setLevel(level)
        ch = logging.StreamHandler()
        ch.setLevel(level)
        fmt = logging.Formatter("[%(asctime)s] %(levelname)s - %(message)s")
        ch.setFormatter(fmt)
        logger.addHandler(ch)
    return logger

LOGGER = setup_logger()

# ------------------------- Configs -------------------------

@dataclass
class QAOAConfig:
    p: int = 3                         # QAOA depth
    optimizer: str = "BFGS"            # 'BFGS' | 'Nelder-Mead'
    maxiter: int = 200
    tol: Optional[float] = None
    eps: Optional[float] = None        # finite-difference step (BFGS)
    xatol: Optional[float] = None      # Nelder-Mead stopping criteria
    runs: int = 5                      # multi-start attempts
    seed: int = 42
    backend_method: str = "statevector"  # 'statevector' | 'density_matrix'
    backend_device: str = "GPU"          # 'AUTO' | 'CPU' | 'GPU'
    measure_shots: int = 0               # 0 means Statevector expectation

@dataclass
class GraphConfig:
    n: int = 6
    d: int = 3    # degree for random regular graph
    ensure_connected: bool = True

@dataclass
class SweepConfig:
    depths: List[int] = None
    repetition_costs: List[float] = None

    def __post_init__(self):
        if self.depths is None:
            self.depths = list(range(1, 11))
        if self.repetition_costs is None:
            self.repetition_costs = list(np.logspace(5, 9, num=12, base=10))

# ------------------------- GPU utils -------------------------

def init_backend(method: str, device_pref: str) -> AerSimulator:
    """
    Initialize AerSimulator with GPU if requested/available, else fallback to CPU.
    """
    if device_pref.upper() == "GPU":
        try:
            os.environ["QISKIT_AER_USE_CUDA"] = "1"
            backend = AerSimulator(method=method, device="GPU")
            LOGGER.info("AerSimulator on GPU initialized.")
            return backend
        except Exception as e:
            LOGGER.warning(f"GPU init failed ({e}). Falling back to CPU.")
            return AerSimulator(method=method, device="CPU")
    elif device_pref.upper() == "AUTO":
        # Try GPU first, then CPU
        try:
            os.environ["QISKIT_AER_USE_CUDA"] = "1"
            backend = AerSimulator(method=method, device="GPU")
            LOGGER.info("AerSimulator AUTO -> using GPU.")
            return backend
        except Exception as e:
            LOGGER.info(f"GPU not available ({e}). Using CPU.")
            return AerSimulator(method=method, device="CPU")
    else:
        backend = AerSimulator(method=method, device="CPU")
        LOGGER.info("AerSimulator on CPU initialized.")
        return backend

# ------------------------- Graph utilities -------------------------

def generate_random_regular_graph(cfg: GraphConfig, rng: np.random.Generator) -> nx.Graph:
    while True:
        G = nx.random_regular_graph(d=cfg.d, n=cfg.n, seed=int(rng.integers(1, 10_000)))
        if not cfg.ensure_connected or nx.is_connected(G):
            return G

def graph_maxcut_cost(bitstring: str, edges: List[Tuple[int, int]]) -> float:
    cut = 0
    for u, v in edges:
        cut += 1 if bitstring[u] != bitstring[v] else 0
    return cut / len(edges) if len(edges) > 0 else 0.0

# ------------------------- QAOA circuit -------------------------

def build_qaoa_circuit(params: np.ndarray, p: int, n_qubits: int, edges: List[Tuple[int, int]]) -> QuantumCircuit:
    qc = QuantumCircuit(n_qubits)
    gammas = params[:p]
    betas  = params[p:]

    qc.h(range(n_qubits))

    for layer in range(p):
        gamma = gammas[layer]
        beta  = betas[layer]

        # Cost layer (Ising ZZ terms via CX-RZ-CX)
        for (i, j) in edges:
            qc.cx(i, j)
            qc.rz(2.0 * gamma, j)
            qc.cx(i, j)

        # Mixer layer
        for i in range(n_qubits):
            qc.rx(2.0 * beta, i)

    return qc

# ------------------------- Executor -------------------------

class QAOAExecutor:
    def __init__(self, cfg: QAOAConfig):
        self.cfg = cfg
        self.backend = init_backend(cfg.backend_method, cfg.backend_device)
        LOGGER.info(f"Backend options: {self.backend.options}")

    def expectation_fraction_satisfied(self, qc: QuantumCircuit, edges: List[Tuple[int, int]]) -> float:
        # Statevector path for exact expectation
        if self.cfg.measure_shots == 0 and self.cfg.backend_method == "statevector":
            sv = Statevector.from_instruction(qc)
            probs = sv.probabilities_dict()
            total = 0.0
            for bitstring, p in probs.items():
                total += p * graph_maxcut_cost(bitstring, edges)
            return total
        # Sampling path
        circ = qc.copy()
        circ.measure_all()
        job = self.backend.run(circ, shots=max(1, self.cfg.measure_shots))
        result = job.result()
        counts = result.get_counts()
        shots = sum(counts.values())
        total = 0.0
        for bitstring, c in counts.items():
            total += (c / shots) * graph_maxcut_cost(bitstring, edges)
        return total

# ------------------------- Optimization -------------------------

def optimize_qaoa(executor: QAOAExecutor,
                  p: int,
                  n_qubits: int,
                  edges: List[Tuple[int, int]],
                  optimizer: str,
                  maxiter: int,
                  tol: Optional[float] = None,
                  eps: Optional[float] = None,
                  xatol: Optional[float] = None,
                  runs: int = 3,
                  rng: Optional[np.random.Generator] = None) -> Dict:
    if rng is None:
        rng = np.random.default_rng(42)

    best_val = -np.inf
    best_params = None
    all_vals = []

    def objective(x):
        qc = build_qaoa_circuit(x, p, n_qubits, edges)
        val = executor.expectation_fraction_satisfied(qc, edges)
        return -val  # maximize satisfaction -> minimize negative

    for r in range(runs):
        init_params = rng.uniform(0.0, math.pi, size=2 * p)
        if optimizer == "BFGS":
            res = minimize(objective, init_params, method="BFGS",
                           options={"maxiter": maxiter, "eps": eps} if eps is not None else {"maxiter": maxiter},
                           tol=tol)
        elif optimizer == "Nelder-Mead":
            res = minimize(objective, init_params, method="Nelder-Mead",
                           options={"maxiter": maxiter, "xatol": xatol} if xatol is not None else {"maxiter": maxiter},
                           tol=tol)
        else:
            raise ValueError(f"Unsupported optimizer: {optimizer}")

        val = -res.fun
        all_vals.append(val)
        if val > best_val:
            best_val = val
            best_params = res.x

        LOGGER.info(f"Run {r+1}/{runs} -> val={val:.4f}, success={res.success}, iters={res.nit}")

    return {"best_value": best_val, "best_params": best_params, "all_values": all_vals}

# ------------------------- Sweeps & analysis -------------------------

def smooth_non_decreasing(vals: List[float], window: int = 3) -> np.ndarray:
    smoothed = uniform_filter1d(vals, size=window)
    return np.maximum.accumulate(smoothed)

def sweep_depths(executor: QAOAExecutor,
                 graph: nx.Graph,
                 cfg: QAOAConfig,
                 depths: List[int],
                 optimizer_label: str) -> Dict:
    n_qubits = graph.number_of_nodes()
    edges = list(graph.edges)
    series = []
    for p in depths:
        res = optimize_qaoa(
            executor=executor,
            p=p,
            n_qubits=n_qubits,
            edges=edges,
            optimizer=cfg.optimizer,
            maxiter=cfg.maxiter,
            tol=cfg.tol,
            eps=cfg.eps,
            xatol=cfg.xatol,
            runs=cfg.runs,
            rng=np.random.default_rng(cfg.seed + p)
        )
        series.append(res["best_value"])
        LOGGER.info(f"[Depth {p}] {optimizer_label} best={res['best_value']:.4f}")
    return {"depths": depths, "values": series, "label": optimizer_label}

def sweep_repetition_costs(executor: QAOAExecutor,
                           graph: nx.Graph,
                           cfg: QAOAConfig,
                           repetition_costs: List[float],
                           optimizer_label: str) -> Dict:
    n_qubits = graph.number_of_nodes()
    edges = list(graph.edges)
    vals = []
    for i, cost in enumerate(repetition_costs):
        # Map repetition cost to iteration budget (proxy)
        scaled_iters = int(min(cfg.maxiter, max(20, math.log10(cost) * 20)))
        temp_cfg = QAOAConfig(**asdict(cfg))
        temp_cfg.maxiter = scaled_iters
        res = optimize_qaoa(
            executor=executor,
            p=temp_cfg.p,
            n_qubits=n_qubits,
            edges=edges,
            optimizer=temp_cfg.optimizer,
            maxiter=temp_cfg.maxiter,
            tol=temp_cfg.tol,
            eps=temp_cfg.eps,
            xatol=temp_cfg.xatol,
            runs=temp_cfg.runs,
            rng=np.random.default_rng(cfg.seed + i)
        )
        vals.append(res["best_value"])
        LOGGER.info(f"[Cost {cost:.2e}] {optimizer_label} best={res['best_value']:.4f} (iters={scaled_iters})")
    return {"costs": repetition_costs, "values": vals, "label": optimizer_label}

# ------------------------- Persistence & plotting -------------------------

def save_json(obj: Dict, path: str):
    os.makedirs(os.path.dirname(path), exist_ok=True)
    with open(path, "w") as f:
        json.dump(obj, f, indent=2)

def plot_depth_series(depths, values, smoothed, label, outdir):
    plt.figure(figsize=(10, 6))
    plt.plot(depths, values, marker='o', linewidth=1.5, color='steelblue', label=f"{label} (raw)")
    plt.plot(depths, smoothed, marker='s', linewidth=2.0, color='darkblue', label=f"{label} (smoothed)")
    plt.xlabel("QAOA depth p")
    plt.ylabel("Fraction of satisfied cut edges")
    plt.title("Satisfaction vs QAOA depth (p)")
    plt.grid(True, linestyle='--', alpha=0.5)
    plt.legend()
    os.makedirs(outdir, exist_ok=True)
    path = os.path.join(outdir, "depth_series.png")
    plt.tight_layout()
    plt.savefig(path, dpi=180)
    plt.close()
    return path

def plot_cost_series(costs, values, smoothed, label, outdir):
    plt.figure(figsize=(10, 6))
    plt.semilogx(costs, values, marker='o', linewidth=1.5, color='orange', label=f"{label} (raw)")
    plt.semilogx(costs, smoothed, marker='s', linewidth=2.0, color='darkorange', label=f"{label} (smoothed)")
    plt.xlabel("Repetition cost (log scale)")
    plt.ylabel("Fraction of satisfied cut edges")
    plt.title("Satisfaction vs repetition cost")
    plt.grid(True, which='both', linestyle='--', alpha=0.5)
    plt.legend()
    os.makedirs(outdir, exist_ok=True)
    path = os.path.join(outdir, "cost_series.png")
    plt.tight_layout()
    plt.savefig(path, dpi=180)
    plt.close()
    return path

# ------------------------- Defaults & run-all -------------------------

def default_configs() -> Tuple[QAOAConfig, GraphConfig, SweepConfig]:
    qcfg = QAOAConfig(
        p=3, optimizer="BFGS", maxiter=200, eps=0.01, runs=5,
        seed=42, backend_method="statevector", backend_device="AUTO", measure_shots=0
    )
    gcfg = GraphConfig(n=6, d=3, ensure_connected=True)
    scfg = SweepConfig(depths=list(range(1, 11)),
                       repetition_costs=list(np.logspace(5, 9, num=12, base=10)))
    return qcfg, gcfg, scfg

def run_all(qcfg: QAOAConfig, gcfg: GraphConfig, scfg: SweepConfig, outdir: str = "artifacts") -> Dict:
    rng = np.random.default_rng(qcfg.seed)
    graph = generate_random_regular_graph(gcfg, rng)
    executor = QAOAExecutor(qcfg)

    depth_series = sweep_depths(executor, graph, qcfg, scfg.depths, optimizer_label=qcfg.optimizer)
    depth_series["values_smoothed"] = smooth_non_decreasing(depth_series["values"]).tolist()

    cost_series = sweep_repetition_costs(executor, graph, qcfg, scfg.repetition_costs, optimizer_label=qcfg.optimizer)
    cost_series["values_smoothed"] = smooth_non_decreasing(cost_series["values"]).tolist()

    result = {
        "config": {"qaoa": asdict(qcfg), "graph": asdict(gcfg), "sweep": asdict(scfg)},
        "graph": {"nodes": graph.number_of_nodes(), "edges": list(graph.edges())},
        "depth_series": depth_series,
        "cost_series": cost_series
    }

    timestamp = time.strftime("%Y%m%d-%H%M%S")
    json_path = os.path.join(outdir, f"qaoa_results_{timestamp}.json")
    save_json(result, json_path)
    LOGGER.info(f"Saved results to {json_path}")

    # Save plots
    dpath = plot_depth_series(depths=depth_series["depths"],
                              values=depth_series["values"],
                              smoothed=depth_series["values_smoothed"],
                              label=depth_series["label"],
                              outdir=outdir)
    cpath = plot_cost_series(costs=cost_series["costs"],
                             values=cost_series["values"],
                             smoothed=cost_series["values_smoothed"],
                             label=cost_series["label"],
                             outdir=outdir)

    # Summary for recruiters
    summary = {
        "best_depth_value": float(np.max(depth_series["values"])),
        "best_cost_value": float(np.max(cost_series["values"])),
        "graph_nodes": result["graph"]["nodes"],
        "graph_edges_count": len(result["graph"]["edges"]),
        "optimizer": qcfg.optimizer,
        "backend_device": qcfg.backend_device
    }
    save_json(summary, os.path.join(outdir, "summary.json"))

    return {"result_json": json_path, "depth_plot": dpath, "cost_plot": cpath, "summary_json": os.path.join(outdir, "summary.json")}

# ------------------------- CLI -------------------------

def parse_args():
    parser = argparse.ArgumentParser(description="Hybrid Quantum–Classical QAOA (Qiskit-only) single-file runner")
    parser.add_argument("--backend-device", type=str, default=None, help="AUTO | CPU | GPU")
    parser.add_argument("--backend-method", type=str, default=None, help="statevector | density_matrix")
    parser.add_argument("--optimizer", type=str, default=None, help="BFGS | Nelder-Mead")
    parser.add_argument("--p", type=int, default=None, help="QAOA depth p")
    parser.add_argument("--runs", type=int, default=None, help="Multi-start runs")
    parser.add_argument("--maxiter", type=int, default=None, help="Max iterations for optimizer")
    parser.add_argument("--eps", type=float, default=None, help="Finite-diff step for BFGS")
    parser.add_argument("--xatol", type=float, default=None, help="Nelder-Mead tolerance")
    parser.add_argument("--shots", type=int, default=None, help="Measurement shots (0 uses Statevector)")
    parser.add_argument("--outdir", type=str, default="artifacts", help="Output directory")
    return parser.parse_args()

def main():
    args = parse_args()
    qcfg, gcfg, scfg = default_configs()

    # Overrides from CLI
    if args.backend_device:
        qcfg.backend_device = args.backend_device
    if args.backend_method:
        qcfg.backend_method = args.backend_method
    if args.optimizer:
        qcfg.optimizer = args.optimizer
    if args.p is not None:
        qcfg.p = args.p
    if args.runs is not None:
        qcfg.runs = args.runs
    if args.maxiter is not None:
        qcfg.maxiter = args.maxiter
    if args.eps is not None:
        qcfg.eps = args.eps
    if args.xatol is not None:
        qcfg.xatol = args.xatol
    if args.shots is not None:
        qcfg.measure_shots = args.shots

    artifacts = run_all(qcfg, gcfg, scfg, outdir=args.outdir)
    print("Artifacts saved:")
    for k, v in artifacts.items():
        print(f" - {k}: {v}")

if __name__ == "__main__":
    main()